In [2]:
import json
import duckdb

## Yapılacaklar

* Metriğinizi iyi belirleyin. (Unutmayın ki her ay sadece abone kitlesinden 1000'de birlik kısmı kaybediyoruz.)

* Modellemenizi gerçekleştirin. Tercihen yorumlaması ve açıklaması kolay olan bir model olsun. Regülasyon sağlayan kurum açıklanması zor modellerde bizi zorlayacaktır.

* Yapacağınız sunumda iş birimimiz olacak. En basit bir şekilde onlara çözümümüzü aktarmamız gerekiyor.

## Veri Alanları

* **id**: Müşteri IDsi
* **age**: Müşterinin yaşı
* **tenure**: Müşterinin operatörde geçirdiği toplam süre (ay cinsinden)
* **service_type**: Ön Ödemeli, Peşin Ödemeli veya Geniş Bant internet müşterisi
* **avg_call_duration**: Ortalama sesli görüşme süresi (saniye)
* **data_usage**: GB Upload + Download
* **roaming_usage**: Ortalama roaming sesli görüşme süresi
* **monthly_charge**: Aylık ortalama fatura
* **overdue_payments**: Ödemesi geçen fatura adedi
* **auto_payment**: Otomatik ödeme talimatı
* **avg_top_up_count**: Ön yüklemeli abone için aylık yükleme sayısı ortalaması
* **call_drops**: Şebekede yaşadığı sesli görüşme kesilmesi
* **customer_support_calls**: Toplam çağrı merkezi araması
* **satisfaction_score**: Müşteri çağrı merkezi değerlendirme skoru 
* **apps**: Müşterinin kullandığı diğer servislerimiz
    * İzleGo 
    * RitimGo
    * CüzdanX
    * HızlıPazar
    * Konuşalım
* **churn**: bool

In [3]:
db_path = "../Data/capstone.db"

In [4]:
with duckdb.connect(db_path) as con:
    con.execute("create table if not exists churn as select * from read_json('../Data/*.jsonl')")

In [5]:
with duckdb.connect(db_path) as con:
    con.sql("select * from churn").show()

┌──────────────────────────────────────┬───────┬────────┬──────────────┬───────────────────┬────────────┬───────────────┬────────────────┬──────────────────┬──────────────┬──────────────────┬────────────┬────────────────────────┬────────────────────┬─────────────┬─────────┐
│                  id                  │  age  │ tenure │ service_type │ avg_call_duration │ data_usage │ roaming_usage │ monthly_charge │ overdue_payments │ auto_payment │ avg_top_up_count │ call_drops │ customer_support_calls │ satisfaction_score │    apps     │  churn  │
│                 uuid                 │ int64 │ int64  │   varchar    │      double       │   double   │    double     │     double     │      int64       │   boolean    │      int64       │   int64    │         int64          │       double       │  varchar[]  │ boolean │
├──────────────────────────────────────┼───────┼────────┼──────────────┼───────────────────┼────────────┼───────────────┼────────────────┼──────────────────┼──────────────┼───

In [6]:
with duckdb.connect(db_path) as con:
    df = con.sql("select * from churn").df()

In [7]:
#df.head()

In [8]:
#df.info()

In [9]:
#df.describe().T

In [10]:
apps = ['İzleGo', 'RitimGo', 'CüzdanX', 'HızlıPazar', 'Konuşalım']
df[apps] = 0

In [11]:
for idx, app_list in enumerate(df["apps"]):
    for app in app_list:
        if app in apps:
            df.loc[idx,app] = 1
    

In [70]:
columns = list(df.columns)
columns.remove("apps")

In [71]:
def analyze_data(dataframe, cat_th=10, car_th=20):
    """
    It gives the names of categorical, numerical and categorical but cardinal variables in the data set. It also performs incomplete data analysis.
    Parameters
    ------
        dataframe: dataframe
            The dataframe from which variable names are to be retrieved
        cat_th: int, optional
            Class threshold value for numeric but categorical variables
        car_th: int, optional
            Class threshold for categorical but cardinal variables

    Returns
    ------
        cat_cols: list
            Categorical variable list
        num_cols: list
            Numerik değişken listesi
        cat_but_car: list
            Categorical view cardinal variable list
    """
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == "O"]
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype != "O"]

    num_but_cat = [col for col in num_cols if dataframe[col].nunique() < cat_th]
    cat_but_car = [col for col in cat_cols if dataframe[col].nunique() > car_th]

    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    cat_cols = cat_cols + num_but_cat
    
    print(f"Number of Observations: {dataframe.shape[0]}")
    print(f"Number of Variables: {dataframe.shape[1]}")
    print(f'Cat cols: {len(cat_cols)}, Num cols: {len(num_cols)}, Cat but car cols: {len(cat_but_car)}')

    return cat_cols, num_cols, cat_but_car

In [72]:
cat_cols, num_cols, cat_but_car = analyze_data(df[columns])

Number of Observations: 10000000
Number of Variables: 20
Cat cols: 9, Num cols: 10, Cat but car cols: 1


In [73]:
df[cat_but_car].isna().sum()

id    0
dtype: int64

In [1]:
with duckdb.connect(db_path) as con:
    con.sql("select disticts Prepaid from df").show()

NameError: name 'duckdb' is not defined